<a href="https://colab.research.google.com/github/zareee12/Data_Engineer_Streamlit_Airflow/blob/main/Batch_Processing_dengan_Pyspark_Zare.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Instalasi

In [2]:
!java -version


openjdk version "11.0.20.1" 2023-08-24
OpenJDK Runtime Environment (build 11.0.20.1+1-post-Ubuntu-0ubuntu122.04)
OpenJDK 64-Bit Server VM (build 11.0.20.1+1-post-Ubuntu-0ubuntu122.04, mixed mode, sharing)


In [3]:
# Install pyspark
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285387 sha256=34574f1bef73c2d1609feabe3bdd31fe38c32f56f03f28b4bdc9ac441457b505
  Stored in directory: /root/.cache/pip/wheels/0d/77/a3/ff2f74cc9ab41f8f594dabf0579c2a7c6de920d584206e0834
Successfully built pyspark


# Mendownload Data

In [4]:
#Download datasets
from IPython.display import clear_output

! wget https://raw.githubusercontent.com/databricks/Spark-The-Definitive-Guide/master/data/retail-data/all/online-retail-dataset.csv

--2023-09-20 04:45:58--  https://raw.githubusercontent.com/databricks/Spark-The-Definitive-Guide/master/data/retail-data/all/online-retail-dataset.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 45038760 (43M) [text/plain]
Saving to: ‘online-retail-dataset.csv’

online-retail-datas 100%[===================>]  42.95M  --.-KB/s    in 0.1s    

2023-09-20 04:45:58 (343 MB/s) - ‘online-retail-dataset.csv’ saved [45038760/45038760]



In [5]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("dibimbingDF").getOrCreate()

#  Operasi Kolom Tingkat Lanjut

## Window Function

In [6]:
# create dataframe
data = [("John", "Sales", 5000),
        ("Sara", "Engineering", 6500),
        ("Chris", "Sales", 5500),
        ("Rachel", "Engineering", 6500),
        ("Maria", "Engineering", 6000),
        ("Eric", "Sales", 4500),
        ("Tom", "Engineering", 7000),
        ("Kim", "Sales", 4000),
        ("Mike", "Engineering", 7500)]

schema = ["name", "department", "salary"]

In [7]:
# add rank and dense_rank columns
from pyspark.sql.functions import rank, dense_rank, ntile, row_number, col
from pyspark.sql.window import Window

df = spark.createDataFrame(data, schema)

w1 = Window().partitionBy("department").orderBy(col("salary").desc())
df = (
    df
    .withColumn("rank", rank().over(w1))
    .withColumn("dense_rank", dense_rank().over(w1))
    .withColumn("ntile", ntile(4).over(w1))
    .withColumn("row_num", row_number().over(w1))
)

In [8]:
df.show()

+------+-----------+------+----+----------+-----+-------+
|  name| department|salary|rank|dense_rank|ntile|row_num|
+------+-----------+------+----+----------+-----+-------+
|  Mike|Engineering|  7500|   1|         1|    1|      1|
|   Tom|Engineering|  7000|   2|         2|    1|      2|
|  Sara|Engineering|  6500|   3|         3|    2|      3|
|Rachel|Engineering|  6500|   3|         3|    3|      4|
| Maria|Engineering|  6000|   5|         4|    4|      5|
| Chris|      Sales|  5500|   1|         1|    1|      1|
|  John|      Sales|  5000|   2|         2|    2|      2|
|  Eric|      Sales|  4500|   3|         3|    3|      3|
|   Kim|      Sales|  4000|   4|         4|    4|      4|
+------+-----------+------+----+----------+-----+-------+



In [9]:
from pyspark.sql.functions import col, lead, lag, unix_timestamp
from pyspark.sql.window import Window

# create example dataset
data = [
    ('customer1', '2022-01-01 00:00:00'),
    ('customer1', '2022-01-02 00:00:00'),
    ('customer1', '2022-01-03 00:00:00'),
    ('customer2', '2022-01-01 00:00:00'),
    ('customer2', '2022-01-04 00:00:00'),
    ('customer2', '2022-01-06 00:00:00'),
]

df = (
    spark
    .createDataFrame(data, ['customer', 'transaction_time'])
    .withColumn('transaction_time_unix', unix_timestamp('transaction_time'))
)

# create window partitioned by customer and ordered by transaction_time
w = Window.partitionBy('customer').orderBy('transaction_time_unix')

(
    df
 .withColumn('prev_transaction_time', lag('transaction_time_unix').over(w))
 .withColumn('next_transaction_time', lead('transaction_time_unix').over(w))
 .withColumn('time_gap', col('transaction_time_unix') - col('prev_transaction_time'))
 .withColumn('next_time_gap', col('next_transaction_time') - col('transaction_time_unix'))
 .show()
)

+---------+-------------------+---------------------+---------------------+---------------------+--------+-------------+
| customer|   transaction_time|transaction_time_unix|prev_transaction_time|next_transaction_time|time_gap|next_time_gap|
+---------+-------------------+---------------------+---------------------+---------------------+--------+-------------+
|customer1|2022-01-01 00:00:00|           1640995200|                 null|           1641081600|    null|        86400|
|customer1|2022-01-02 00:00:00|           1641081600|           1640995200|           1641168000|   86400|        86400|
|customer1|2022-01-03 00:00:00|           1641168000|           1641081600|                 null|   86400|         null|
|customer2|2022-01-01 00:00:00|           1640995200|                 null|           1641254400|    null|       259200|
|customer2|2022-01-04 00:00:00|           1641254400|           1640995200|           1641427200|  259200|       172800|
|customer2|2022-01-06 00:00:00| 

In [10]:
from pyspark.sql.functions import sum, first, last, mean, stddev, window
from pyspark.sql import Window

# create a sample dataframe
sales_data = [("Product A", 100, "2022-01-01 10:00:00", "Location 1"),
              ("Product B", 200, "2022-01-01 11:00:00", "Location 1"),
              ("Product A", 150, "2022-01-02 12:00:00", "Location 2"),
              ("Product C", 300, "2022-01-03 14:00:00", "Location 2"),
              ("Product D", 250, "2022-01-04 15:00:00", "Location 2"),
              ("Product E", 150, "2022-01-05 16:00:00", "Location 3"),
              ("Product F", 180, "2022-01-06 17:00:00", "Location 3"),
              ("Product G", 220, "2022-01-07 18:00:00", "Location 3")]

sales_df = spark.createDataFrame(sales_data, ["Product", "Price", "Sale Time", "Location"])

In [11]:
# define window specification to partition by location and order by sale time
location_window_spec = Window.partitionBy("Location").orderBy("Sale Time")

# calculate total sales revenue for each location over the entire duration of the dataset
total_sales_revenue = sum("Price").over(Window.partitionBy("Location"))

# calculate first and last sale date for each location
first_sale_date = first("Sale Time").over(location_window_spec)
last_sale_date = last("Sale Time").over(location_window_spec)

# calculate average and standard deviation of the sale prices for each location
average_sale_price = mean("Price").over(location_window_spec)
stddev_sale_price = stddev("Price").over(location_window_spec)

# add the calculated columns to the dataframe
sales_df = (
    sales_df
    .withColumn("Total Sales Revenue", total_sales_revenue)
    .withColumn("First Sale Date", first_sale_date)
    .withColumn("Last Sale Date", last_sale_date)
    .withColumn("Average Sale Price", average_sale_price)
    .withColumn("Stddev Sale Price", stddev_sale_price)
)

# show the results
sales_df.show()

+---------+-----+-------------------+----------+-------------------+-------------------+-------------------+------------------+------------------+
|  Product|Price|          Sale Time|  Location|Total Sales Revenue|    First Sale Date|     Last Sale Date|Average Sale Price| Stddev Sale Price|
+---------+-----+-------------------+----------+-------------------+-------------------+-------------------+------------------+------------------+
|Product A|  100|2022-01-01 10:00:00|Location 1|                300|2022-01-01 10:00:00|2022-01-01 10:00:00|             100.0|              null|
|Product B|  200|2022-01-01 11:00:00|Location 1|                300|2022-01-01 10:00:00|2022-01-01 11:00:00|             150.0| 70.71067811865476|
|Product A|  150|2022-01-02 12:00:00|Location 2|                700|2022-01-02 12:00:00|2022-01-02 12:00:00|             150.0|              null|
|Product C|  300|2022-01-03 14:00:00|Location 2|                700|2022-01-02 12:00:00|2022-01-03 14:00:00|          

In [12]:
# group the data by location and window the data by day to get daily aggregated sales data
(
    sales_df
    .groupBy(window("Sale Time", "1 day"))
    .agg(sum("Price").alias("Daily Sales Revenue"))
    .show(truncate=False)
)

+------------------------------------------+-------------------+
|window                                    |Daily Sales Revenue|
+------------------------------------------+-------------------+
|{2022-01-02 00:00:00, 2022-01-03 00:00:00}|150                |
|{2022-01-01 00:00:00, 2022-01-02 00:00:00}|300                |
|{2022-01-03 00:00:00, 2022-01-04 00:00:00}|300                |
|{2022-01-04 00:00:00, 2022-01-05 00:00:00}|250                |
|{2022-01-05 00:00:00, 2022-01-06 00:00:00}|150                |
|{2022-01-06 00:00:00, 2022-01-07 00:00:00}|180                |
|{2022-01-07 00:00:00, 2022-01-08 00:00:00}|220                |
+------------------------------------------+-------------------+



In [13]:
# group the data by location and window the data by day to get daily aggregated sales data
(
    sales_df
    .groupBy("Location", window("Sale Time", "1 day"))
    .agg(sum("Price").alias("Daily Sales Revenue"))
    .show(truncate=False)
)

+----------+------------------------------------------+-------------------+
|Location  |window                                    |Daily Sales Revenue|
+----------+------------------------------------------+-------------------+
|Location 2|{2022-01-03 00:00:00, 2022-01-04 00:00:00}|300                |
|Location 2|{2022-01-02 00:00:00, 2022-01-03 00:00:00}|150                |
|Location 1|{2022-01-01 00:00:00, 2022-01-02 00:00:00}|300                |
|Location 3|{2022-01-06 00:00:00, 2022-01-07 00:00:00}|180                |
|Location 3|{2022-01-05 00:00:00, 2022-01-06 00:00:00}|150                |
|Location 3|{2022-01-07 00:00:00, 2022-01-08 00:00:00}|220                |
|Location 2|{2022-01-04 00:00:00, 2022-01-05 00:00:00}|250                |
+----------+------------------------------------------+-------------------+



# UDF

In [14]:
from pyspark.sql.functions import col, udf, pandas_udf
from pyspark.sql.types import IntegerType

# Create a sample dataframe
df = spark.createDataFrame([(1, "apple"), (2, "banana"), (3, "orange"),
                            (4, "apple"), (5, "banana"), (6, "orange"),
                            (7, "apple"), (8, "banana"), (9, "orange"),
                            (10, "apple")], ["id", "fruit"])

In [15]:
# Define the UDF logic
def string_length(s):
    return len(s)

# Define the Python UDF
string_length_udf = udf(string_length, IntegerType())

# Apply the Python UDF and display the result
df1 = df.withColumn("length", string_length_udf(col("fruit")))
df1.show()

+---+------+------+
| id| fruit|length|
+---+------+------+
|  1| apple|     5|
|  2|banana|     6|
|  3|orange|     6|
|  4| apple|     5|
|  5|banana|     6|
|  6|orange|     6|
|  7| apple|     5|
|  8|banana|     6|
|  9|orange|     6|
| 10| apple|     5|
+---+------+------+



# Pandas UDF


In [16]:
import pandas as pd

# Define the Pandas UDF
@pandas_udf(IntegerType())
def string_length_pandas_udf(s: pd.Series) -> pd.Series:
    return s.str.len()

# Apply the Pandas UDF and display the result
df2 = df.withColumn("length", string_length_pandas_udf(col("fruit")))
df2.show()

+---+------+------+
| id| fruit|length|
+---+------+------+
|  1| apple|     5|
|  2|banana|     6|
|  3|orange|     6|
|  4| apple|     5|
|  5|banana|     6|
|  6|orange|     6|
|  7| apple|     5|
|  8|banana|     6|
|  9|orange|     6|
| 10| apple|     5|
+---+------+------+



# Partition & Bucket

## Data Cleanup



In [17]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, TimestampType, DoubleType
from pyspark.sql.functions import to_timestamp, to_date


schema = StructType([
    StructField("InvoiceNo", StringType(), True),
    StructField("StockCode", StringType(), True),
    StructField("Description", StringType(), True),
    StructField("Quantity", IntegerType(), True),
    StructField("InvoiceDate", StringType(), True),
    StructField("UnitPrice", DoubleType(), True),
    StructField("CustomerID", StringType(), True),
    StructField("Country", StringType(), True)
])

retail_df = (
    spark
    .read.csv('online-retail-dataset.csv',header=True, schema=schema)
    .withColumn("InvoiceTime", to_timestamp("InvoiceDate", "M/d/yyyy H:mm"))
    .withColumn("InvoiceDate", to_date("InvoiceDate", "M/d/yyyy H:mm"))
)
retail_df.show(10)

+---------+---------+--------------------+--------+-----------+---------+----------+--------------+-------------------+
|InvoiceNo|StockCode|         Description|Quantity|InvoiceDate|UnitPrice|CustomerID|       Country|        InvoiceTime|
+---------+---------+--------------------+--------+-----------+---------+----------+--------------+-------------------+
|   536365|   85123A|WHITE HANGING HEA...|       6| 2010-12-01|     2.55|     17850|United Kingdom|2010-12-01 08:26:00|
|   536365|    71053| WHITE METAL LANTERN|       6| 2010-12-01|     3.39|     17850|United Kingdom|2010-12-01 08:26:00|
|   536365|   84406B|CREAM CUPID HEART...|       8| 2010-12-01|     2.75|     17850|United Kingdom|2010-12-01 08:26:00|
|   536365|   84029G|KNITTED UNION FLA...|       6| 2010-12-01|     3.39|     17850|United Kingdom|2010-12-01 08:26:00|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6| 2010-12-01|     3.39|     17850|United Kingdom|2010-12-01 08:26:00|
|   536365|    22752|SET 7 BABUSHKA NE..

## Partition

In [18]:
(
    retail_df
 .write
 .mode("overwrite")
 .partitionBy('InvoiceDate')
 .parquet("/content/spark/online-retail-partitioned")
)

In [19]:
partitioned_table = (
    spark
    .read
    .parquet("/content/spark/online-retail-partitioned")
)

In [20]:
partitioned_table.printSchema()

root
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- UnitPrice: double (nullable = true)
 |-- CustomerID: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- InvoiceTime: timestamp (nullable = true)
 |-- InvoiceDate: date (nullable = true)



In [21]:
partitioned_table.explain(mode="formatted")

== Physical Plan ==
* ColumnarToRow (2)
+- Scan parquet  (1)


(1) Scan parquet 
Output [9]: [InvoiceNo#477, StockCode#478, Description#479, Quantity#480, UnitPrice#481, CustomerID#482, Country#483, InvoiceTime#484, InvoiceDate#485]
Batched: true
Location: InMemoryFileIndex [file:/content/spark/online-retail-partitioned]
ReadSchema: struct<InvoiceNo:string,StockCode:string,Description:string,Quantity:int,UnitPrice:double,CustomerID:string,Country:string,InvoiceTime:timestamp>

(2) ColumnarToRow [codegen id : 1]
Input [9]: [InvoiceNo#477, StockCode#478, Description#479, Quantity#480, UnitPrice#481, CustomerID#482, Country#483, InvoiceTime#484, InvoiceDate#485]




## Bucket

In [22]:
from pyspark.sql.functions import hour


(
    retail_df
 .withColumn("Hour", hour("InvoiceTime"))
 .write
 .mode("overwrite")
 .partitionBy("InvoiceDate")
 .bucketBy(2, "Hour")
 .sortBy("InvoiceTime")
 .option("path", "/content/spark/online-retail-bucketed")
 .saveAsTable("retail_bucketed")
)

Sekarang kita memiliki 2 berkas Parquet, bukan hanya 1.

# Pandas API On Spark

In [23]:
import pyspark.pandas as ps

retail_ps = retail_df.pandas_api()

/usr/local/lib/python3.10/dist-packages/pyspark/pandas/__init__.py:50: UserWarning: 'PYARROW_IGNORE_TIMEZONE' environment variable was not set. It is required to set this environment variable to '1' in both driver and executor sides if you use pyarrow>=2.0.0. pandas-on-Spark will set it for you but it does not work if there is a Spark context already launched.
  warnings.warn(


In [24]:
retail_ps.head()

/usr/local/lib/python3.10/dist-packages/pyspark/sql/pandas/conversion.py:251: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  series = series.astype(t, copy=False)


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,InvoiceTime
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01,2.55,17850,United Kingdom,2010-12-01 08:26:00
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01,3.39,17850,United Kingdom,2010-12-01 08:26:00
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01,2.75,17850,United Kingdom,2010-12-01 08:26:00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01,3.39,17850,United Kingdom,2010-12-01 08:26:00
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01,3.39,17850,United Kingdom,2010-12-01 08:26:00


In [25]:
aggregated_df = retail_ps.groupby("InvoiceDate").agg({"InvoiceNo": "count"}).reset_index()
aggregated_df.columns = ["InvoiceDate", "InvoiceCount"]
aggregated_df["InvoiceDate"] = aggregated_df["InvoiceDate"].astype("datetime64[ns]")

In [27]:
aggregated_df.head()

/usr/local/lib/python3.10/dist-packages/pyspark/sql/pandas/conversion.py:251: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  series = series.astype(t, copy=False)


,InvoiceDate,InvoiceCount
0,2011-01-30,722
1,2011-05-06,2051
2,2011-01-23,879
3,2011-07-07,1940
4,2011-07-18,2234


/usr/local/lib/python3.10/dist-packages/pyspark/sql/pandas/conversion.py:251: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  series = series.astype(t, copy=False)


,InvoiceDate,InvoiceCount
0,2011-01-30,722
1,2011-05-06,2051
2,2011-01-23,879
3,2011-07-07,1940
4,2011-07-18,2234


In [28]:
retail_ps[["UnitPrice"]].plot.hist()